In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
import cv2
import matplotlib as plt
import PIL.Image as Image

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
import tf_keras as tfk
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

2025-04-18 16:00:10.052364: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744992010.352190     377 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744992010.428766     377 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
image_shape = (128,128)
classifier = tfk.Sequential([
    hub.KerasLayer("https://www.kaggle.com/models/google/mobilenet-v2/TensorFlow2/035-128-classification/2", input_shape=image_shape+(3,))
])

2025-04-18 16:00:22.632775: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [3]:
 tiger_shark = Image.open('/kaggle/input/tiger-shark-img/Tiger_shark.jpg').resize(image_shape)

In [4]:
tiger_shark = np.array(tiger_shark)/225
tiger_shark

array([[[0.50222222, 0.66222222, 0.84      ],
        [0.50666667, 0.66222222, 0.84      ],
        [0.50666667, 0.65777778, 0.84      ],
        ...,
        [0.15111111, 0.51111111, 0.72      ],
        [0.15555556, 0.50666667, 0.72      ],
        [0.13333333, 0.49777778, 0.71555556]],

       [[0.51111111, 0.66666667, 0.84444444],
        [0.49333333, 0.65777778, 0.83555556],
        [0.49333333, 0.65333333, 0.83555556],
        ...,
        [0.17777778, 0.53333333, 0.73777778],
        [0.16888889, 0.52444444, 0.73333333],
        [0.15111111, 0.51555556, 0.72444444]],

       [[0.51111111, 0.67555556, 0.84888889],
        [0.50222222, 0.66222222, 0.84      ],
        [0.50222222, 0.65777778, 0.83555556],
        ...,
        [0.16444444, 0.52444444, 0.73333333],
        [0.16444444, 0.52444444, 0.72888889],
        [0.14222222, 0.51555556, 0.72444444]],

       ...,

       [[0.45333333, 0.66222222, 0.75111111],
        [0.45777778, 0.66666667, 0.75555556],
        [0.46666667, 0

In [5]:
tiger_shark.shape

(128, 128, 3)

In [6]:
tiger_shark[np.newaxis,...].shape

(1, 128, 128, 3)

In [7]:
result = classifier.predict(tiger_shark[np.newaxis,...])

1/1 [==============================] - 1s 917ms/step


In [8]:
label_index = np.argmax(result)
label_index

4

In [9]:
image_label = []
with open('/kaggle/input/classes-label/ImageNetLabels.txt','r') as f:
    image_label = f.read().splitlines()
image_label[label_index]

'tiger shark'

In [10]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("puneet6060/intel-image-classification")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/intel-image-classification


In [11]:
train_dir = "/kaggle/input/intel-image-classification/seg_train/seg_train"
test_dir = "/kaggle/input/intel-image-classification/seg_test/seg_test"

In [12]:
classes = os.listdir(test_dir)
print("Classes: ",classes)

Classes:  ['mountain', 'street', 'buildings', 'sea', 'forest', 'glacier']


In [13]:
import pathlib
train_dir = pathlib.Path(train_dir)
test_dir = pathlib.Path(test_dir)
train_dir

PosixPath('/kaggle/input/intel-image-classification/seg_train/seg_train')

In [14]:
len(list(train_dir.glob('*/*.jpg')))

14034

In [15]:
image_dict = {
    'mountain': list(train_dir.glob('mountain/*')),
    'street': list(train_dir.glob('street/*')),
    'buildings': list(train_dir.glob('buildings/*')),
    'sea': list(train_dir.glob('sea/*')),
    'forest': list(train_dir.glob('forest/*')),
    'glacier': list(train_dir.glob('glacier/*'))
}

In [16]:
image_label_dict={
    'mountain': 0,
    'street':1,
    'buildings': 2,
    'sea':3,
    'forest':4,
    'glacier':5
 }

In [17]:
dir_image ="/kaggle/input/intel-image-classification/seg_test/seg_test/"

In [18]:
x, y = [], []

for classes, images in image_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        if img is not None:
            img_resize = cv2.resize(img, (128, 128))
            x.append(img_resize)
            y.append(image_label_dict[classes])


In [19]:
xtest, ytest = [], []

for classes, images in image_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        if img is not None:
            img_resize = cv2.resize(img, (128, 128))
            xtest.append(img_resize)
            ytest.append(image_label_dict[classes])

In [20]:
x_train = np.array(x,dtype=float)/255.0
x_test = np.array(xtest,dtype=float)/255.0
y_train = np.array(y)
y_test = np.array(ytest)

In [21]:
mobil = "https://www.kaggle.com/models/google/mobilenet-v2/TensorFlow2/035-128-classification/2"
image_shape = (128,128)
Pretrain_model_without_top_layer = tfk.Sequential([hub.KerasLayer(mobil,input_shape=image_shape+(3,),trainable=False)])

In [25]:
model = tfk.Sequential([
    Pretrain_model_without_top_layer,
    tfk.layers.Dense(6)
])

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_1 (Sequential)   (None, 1001)              1692489   
                                                                 
 dense_1 (Dense)             (None, 6)                 6012      
                                                                 
Total params: 1698501 (6.48 MB)
Trainable params: 6012 (23.48 KB)
Non-trainable params: 1692489 (6.46 MB)
_________________________________________________________________


In [26]:
model.compile(
    optimizer='adam',
             loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics = ['accuracy']
)

In [28]:
model.fit(x_train,y_train,epochs=10)

Epoch 1/10
439/439 [==============================] - 29s 67ms/step - loss: 0.2650 - accuracy: 0.9047
Epoch 2/10
439/439 [==============================] - 30s 67ms/step - loss: 0.2553 - accuracy: 0.9073
Epoch 3/10
439/439 [==============================] - 30s 67ms/step - loss: 0.2486 - accuracy: 0.9084
Epoch 4/10
439/439 [==============================] - 30s 68ms/step - loss: 0.2479 - accuracy: 0.9100
Epoch 5/10
439/439 [==============================] - 30s 69ms/step - loss: 0.2384 - accuracy: 0.9112
Epoch 6/10
439/439 [==============================] - 31s 70ms/step - loss: 0.2366 - accuracy: 0.9132
Epoch 7/10
439/439 [==============================] - 30s 68ms/step - loss: 0.2303 - accuracy: 0.9157
Epoch 8/10
439/439 [==============================] - 30s 68ms/step - loss: 0.2316 - accuracy: 0.9139
Epoch 9/10
439/439 [==============================] - 31s 70ms/step - loss: 0.2242 - accuracy: 0.9152
Epoch 10/10
439/439 [==============================] - 30s 69ms/step - loss: 0.219